<a href="https://colab.research.google.com/github/eroberam/03MIAR---Algoritmos-de-Optimizacion/blob/main/AG2_Roberto_Alcaraz_Machado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre: Roberto Alcaraz Machado <br>
Link:https://colab.research.google.com/drive/1OddXyWsxrzc1keYiOGu70QMHSd6mZ5Ok?usp=sharing <br>
Url: https://github.com/eroberam/03MIAR---Algoritmos-de-Optimizacion

In [1]:
import math

#Viaje por el rio

In [13]:
TARIFAS = [
    [0, 5, 4, 3, float("inf"), 999, 999], # desde nodo 0
    [999, 0, 999, 2, 3, 999, 11], # desde nodo 1
    [999, 999, 0, 1, 999, 4, 10], # desde nodo 2
    [999, 999, 999, 0, 5, 6, 9],
    [999, 999, 999, 999, 0, 999, 4],
    [999, 999, 999, 999, 999, 0, 3],
    [999, 999, 999, 999, 999, 999, 0]
]

def Precios(TARIFAS):
##############################################
  #Total de Nodos
  N = len(TARIFAS[0])

  #Inicialización de la tabla de precios
  PRECIOS = [ [9999]*N for i in [9999]*N ] #n x n
  RUTA = [ [""]*N for i in [""]*N ]

  #Se recorren todos los nodos con dos bucles (origen - destino)
  # para ir construyendo la matriz de PRECIOS
  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i

      for k in range(i, j):
        if PRECIOS[i][k] + TARIFAS [k][j] < MIN:
          MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j])
          RUTA[i][j] = k
        PRECIOS[i][j] = MIN
  return PRECIOS, RUTA

PRECIOS, RUTA = Precios(TARIFAS)
print(PRECIOS)
print(RUTA)

[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, 999, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


In [14]:
# Calculo de la ruta usando la matriz RUTA
def calcular_ruta(RUTA, desde, hasta):
  if desde == RUTA[desde][hasta]:
    # if desde == hasta:
      # print ("Ir a :" + str(desde))
      return desde
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) + ',' + str(RUTA[desde][hasta])

print("\nLa ruta es:")
calcular_ruta(RUTA, 0, 6)


La ruta es:


'0,2,5'

### Ramificación y poda

In [15]:
COSTES = [[11, 12, 18, 40],
          [14, 15, 13, 22],
          [11, 17, 19, 26],
          [17, 14, 20, 28]]

In [20]:
def valor(S, COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]
  return VALOR

def CI(S, COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimación
  for i in range (len(S), len(COSTES)):
    VALOR += min([COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR

def CS(S, COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimación
  for i in range (len(S), len(COSTES)):
    VALOR += max([COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR

In [25]:
# Genera tantos hijos como posibilidades haya para el siguiente elemento de la tupla
# (0,) -> (0, 1), (0, 2), (0, 3)

def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO + (i,)})
  return HIJOS
crear_hijos((0,1, 3),4)

[{'s': (0, 1, 3, 2)}]

In [27]:
def ramificacion_y_poda(COSTES):
  #Construcción iterativa de soluciones(arbol). En cada etapa asignamos un agente (ramas).
  #Nodos del grafo {s:(1, 2), CI:3, CS:5}
  #print(COSTES)
  DIMENSION = len(COSTES)
  MEJOR_SOLUCION = tuple(i for i in range(len(COSTES)))
  CotaSup = valor(MEJOR_SOLUCION, COSTES)
  #print("Cota Superopr:", CotaSup)

  NODOS = []
  NODOS.append({'s':(), 'ci':CI((), COSTES)})

  iteracion = 0
  while (len(NODOS) > 0):
    iteracion += 1
    nodo_prometedor = [min(NODOS, key=lambda x:x['ci'])][0]['s']
    #print('Nodo prometedor:', nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    HIJOS = [{'s':x['s'], 'ci':CI(x['s'], COSTES)} for x in crear_hijos(nodo_prometedor, DIMENSION)]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
    if len(NODO_FINAL) > 0:
      #print("\n***********Soluciones:", [x for x in HIJOS if len(x['s']) == DIMENSION])
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL

    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]

    #Añadaimos los hijos
    NODOS.extend(HIJOS)

    #Eliminados el nodo ramificado
    NODOS = [x for x in NODOS if x['s'] != nodo_prometedor]

  print("La solución final es:", MEJOR_SOLUCION, "en ", iteracion, "iteraciones", "para dimension: ", DIMENSION)

ramificacion_y_poda(COSTES)

La solución final es: [{'s': (1, 2, 0, 3), 'ci': 64}] en  10 iteraciones para dimension:  4
